In [1]:
import sqlite3

# the SQLite database is a single file, which is saved on your computer
# To create a database, use the connect method provided by the sqlite3 module
conn = sqlite3.connect('database_0.db')
#Remember that the connect method creates a database only if 
# it cannot find a database in the given location

SQL is a Structured Query Language for creating, modifying, and managing relational databases. It’s used by the most popular database management systems such as MySQL, PostgreSQL, and our favorite SQLite. The SQL language was developed in the 70s by IBM. Over the years, SQL has been modified by many companies that have implemented it in their products. Therefore, it became necessary to introduce a standard that would standardize its syntax.

In [8]:
# Very useful method called 'cursor'. 
# The method creates a Cursor object that allows any SQL statements to be executed in the database.
conn = sqlite3.connect('todo.db')
c = conn.cursor()

# The execute method takes any single SQL statement 
# and optional parameters necessary to execute the query.
c.execute('''
            CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            priority INTEGER NOT NULL
            );'''
          )
# adding 'IF NOT EXISTS' we avoid errors re-running the cell.
# Without it would raise the error 'sqlite3.OperationError: table tasks already exists'


### Inserting Data

In [ ]:
# The INSERT INTO statement is used to insert records in a table. 
# Its syntax is as follows:
'''
INSERT INTO table_name (column1, column2, column3, ..., columnN)
VALUES (value1, value2, value3, ..., valueN);
'''
# example: 
# INSERT INTO tasks (id, name, priority) VALUES (1, 'My first task', 1);
# or INSERT INTO tasks (name, priority) VALUES ('My first task', 1);
# Omitting the ID (primary key) we use the auto-incrementation

# Short form: INSERT INTO table_name VALUES (value1, value2, value3, ..., valueN);
c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', ('My first task', 1))

# The mysterious characters ? used in the INSERT INTO statement 
# are query parameters that are replaced with the correct values 
# during the execution of the statement. 
# In the above example, the first character ? will be replaced with My first task, 
# while the second will be replaced with l. 
#This is to avoid an SQL injection attack in which malicious SQL is appended to a query 
# that could possibly destroy our database.

# All we're missing is to call the commit method provided by the Connection object.
# The commit method confirms our changes (the current transaction).
# If you forget to call it, your changes won't be visible in the database.
conn.commit()
conn.close() # closes the database connection

In [12]:
# The executemany method allows you to insert multiple records at once.
conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')
tasks = [
    ('My first task', 1),
    ('My second task', 5),
    ('My third task', 10),
]
c.executemany('INSERT INTO tasks (name, priority) VALUES (?,?)', tasks)
conn.commit()
conn.close()

### Refactoring

Very important process during software development. 
The main purpose of refactoring is to improve the quality of the code.

Refactoring consists of creating a function containing repetitive fragments.

In [ ]:
class Todo:
    def __init__(self):
        # creating a class called TODO that will connect to the database in the constructor. 
        # If you want, you can implement a separate method called connect for this purpose
        self.conn = sqlite3.connect('todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        # code responsible for creating the taskstable
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def add_task(self):
        # will get the task name and priority from the user instead of using hardcoded values
        name = input('Enter task name: ')
        priority = int(input('Enter priority: '))
        
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
        self.conn.commit()

app = Todo()
app.add_task()

### Reading Data

So far, we haven't displayed any information on the screen about the inserted tasks.
The appropriate SQL statement is named `SELECT`, which allows you to read data from one or more tables.

In [ ]:
conn = sqlite3.connect('todo.db')
c = conn.cursor()
# After the execute method the SELECT statement, the Cursor object is treated as an iterator
for row in c.execute('SELECT * FROM tasks'):
    print(row)
conn.close()

(1, 'My first task', 1)
(2, 'My second task', 5)
(3, 'My third task', 10)
(4, 'My first task', 1)
(5, 'My second task', 5)
(6, 'My third task', 10)
(7, 'ciao', 4)


In [ ]:
conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('SELECT * FROM tasks')
rows = c.fetchall() # to avoid the Cursor to be treated as an iterator
# returns an empty list when no rows are available.
for row in rows:
    print(row)

# Alternative: still no iterator, but one line
row = c.fetchone()
print(row)

conn.close()

### Lab

Our TODO application requires you to add a little security and display the data saved in the database. 

Your task is to implement the following functionalities:
- Create a find_task method, which takes the task name as its argument. The method should return the record found or None otherwise.
- Block the ability to enter an empty task (the name cannot be an empty string).
- Block the ability to enter a task priority less than 1.
- Use the find_task method to block the ability to enter a task with the same name.
- Create a method called show_tasks, responsible for displaying all tasks saved in the database.


In [3]:
class Todo:
    def __init__(self):
        # creating a class called TODO that will connect to the database in the constructor. 
        # If you want, you can implement a separate method called connect for this purpose
        self.conn = sqlite3.connect('todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        # code responsible for creating the taskstable
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def add_task(self):
        # will get the task name and priority from the user instead of using hardcoded values
        task_name = input('Enter task name: ')
        if len(task_name)==0:
            raise ValueError('No Empty Task Name')

        if self.find_task(task_name) is not None:
            raise ValueError('Task Name already used')
        
        priority = int(input('Enter priority: '))
        if priority<1:
            raise ValueError('Only strictly positive values for Priority')
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (task_name, priority))
        self.conn.commit()

    def find_task(self, task_name):
        query = 'SELECT * FROM tasks WHERE name = ?'
        self.c.execute(query, (task_name,))
        rows = self.c.fetchone()
        return rows

    def show_tasks(self):
        # displaying all tasks saved in the database
        self.c.execute('SELECT * FROM tasks')
        rows = self.c.fetchall() 
        for row in rows:
            print(row)

app = Todo()
# app.show_tasks()
app.add_task()
app.show_tasks()

(1, 'My first task', 20)
(2, 'My second task', 5)
(3, 'My third task', 10)
(4, 'My first task', 1)
(5, 'My second task', 5)
(6, 'My third task', 10)
(7, 'ciao', 4)
(8, 'aaa', 3)
(9, 'aaa', 3)
(10, 'ciao', 3)
(11, 'bbb', 0)
(12, 'a', -1)
(13, 'c', 2)
(14, '2', 5)


### Updating Data

In [2]:
conn = sqlite3.connect('todo.db')
c = conn.cursor()

# The UPDATE statement is used to modify existing records in the database.
# NOTE: If you forget about the WHERE clause, all data in the table will be updated.
c.execute('UPDATE tasks SET priority = ? WHERE id = ?', (20, 1))
conn.commit()
c.close()

### Lab 2

The application is almost ready. Let's add the missing functionalities to it:
- Create a method called change_priority, responsible for updating task priority. The method should get the id of the task from the user and its new priority (greater than or equal to 1).
- Create a method called delete_task, responsible for deleting single tasks. The method should get the task id from the user.
- Implement a simple menu consisting of the following options:
    1. Show Tasks (calls the show_tasks method)
    2. Add Task (calls the add_task method)
    3. Change Priority (calls the change_priority method)
    4. Delete Task (calls the delete_task method)
    5. Exit (interrupts program execution)

The program should obtain one of these options from the user, and then call the appropriate method of the TODO object. Choosing option 5 must terminate the program. A menu should be displayed in an infinite loop so that the user can choose an option multiple times.
